**This CGAN is modified from the DCGAN in this PyTorch Tutorial.** Link: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

In [ ]:
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import utils, datasets, transforms
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
torch.manual_seed(0)

**Hyperparameter**

In [ ]:
# Root directory for dataset
dataroot = "content/mnist"

# Number of workers for dataloader
workers = 10

# Batch size during training
batch_size = 100

# Spatial size of training images. All images will be resized to this size using a transformer.
image_size = 32

# Number of channels in the training images. For color images this is 3
nc = 1

# Number of classes in the training images. For mnist dataset this is 10
num_classes = 10

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 10

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1


**Load Data**

In [ ]:
train_data = datasets.MNIST(
    root=dataroot,
    train=True,
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]),
    download=True
)
test_data = datasets.MNIST(
    root=dataroot,
    train=False,
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
)
dataset = train_data+test_data
print(f'Total Size of Dataset: {len(dataset)}')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting content/mnist/MNIST/raw/train-images-idx3-ubyte.gz to content/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting content/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to content/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting content/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to content/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting content/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to content/mnist/MNIST/raw

Total Size of Dataset: 70000


In [ ]:
dataloader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=workers
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')
print(device)

cpu


**Weight Initialization**

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)